In [ ]:
from osgeo import gdal, ogr, osr
import os
import glob

import numpy as np

In [ ]:
driver = ogr.GetDriverByName('KML')
is_overwrite = False

In [ ]:
def convert_to_shp(points, is_overwrite=False):
    if points.endswith('.kml'):
        new_points=points.replace('.kml','.shp')
        if not os.path.exists(new_points) or is_overwrite:
            srcDS = gdal.OpenEx(points)
            ds = gdal.VectorTranslate(new_points, srcDS, format='ESRI Shapefile')
            ds = None
            points = new_points
    return points 

'''
def keep_name_only(points):
    dataSource = ogr.Open(points, 1) 

    layer = dataSource.GetLayer()

    lyrdf = layer.GetLayerDefn()

    id_Name = lyrdf.GetFieldIndex('Name')
    attr_N = lyrdf.GetFieldCount()
    print(attr_N, id_Name)
    for i in range(attr_N):
        if not i == id_Name:
            layer.DeleteField(i)   
    attr_N = lyrdf.GetFieldCount()
    print(attr_N)
    dataSource = None
'''

def split_kml_folders(filename, dst_path, checkonly=False, as_shp=True):
    ref_ = [x for x in filename.split('/') if x.startswith('T') and '_' in x][0]
    tile, crop, person = ref_.replace('.kml','').split('_')
    print(tile, crop, person)
    ds = gdal.OpenEx(filename)
    for i in range(ds.GetLayerCount()):
        layer = ds.GetLayerByIndex(i)
        layer_name = layer.GetDescription()
        # print(layer_name, layer.GetFeatureCount(),)
        if 'group' not in layer_name and layer.GetFeatureCount() > 0:
            layer_name = filename.split('/')[-2]
            assert 'group' in layer_name, filename
        if True:
        # if 'group' in layer_name:     
            sr = layer.GetSpatialRef()  # Spatial Reference
            new_feat = ogr.Feature(layer.GetLayerDefn())  # Dummy feature
            for feat in layer:
                if feat.GetField(0) is None:
                    print(layer_name, filename, person, filename.split('/')[-1])
                    feat_name = filename.split('/')[-1]
                    feat_name = feat_name.replace('.kml','')
                else:
                    
                    feat_name = feat.GetField(0).replace('/','_')

                newdir_ = f"{dst_path}/{tile}/{crop}_{layer_name}_{person}"
                # print(newdir_)
                if not checkonly:
                    if not os.path.exists(newdir_): os.makedirs(newdir_)
                    filename_ = f"{newdir_}/{feat_name}.kml"
                    # print(filename_)
                    # if False:
                    if not os.path.isfile(filename_) or is_overwrite:
                        new_ds = ogr.GetDriverByName('KML').CreateDataSource(filename_)
                        assert new_ds is not None, (filename_,feat_name)
                        new_lyr = new_ds.CreateLayer(feat_name, sr, ogr.wkbPolygon)  # You have to specify the geometry type the layer will contain here with an ogr constant. I assume it is polygon but it can be changed.
                        geom = feat.geometry().Clone()
                        new_feat.SetGeometry(geom)
                        new_lyr.CreateFeature(new_feat)

                        del new_ds, new_lyr
                    if as_shp:
                        x = convert_to_shp(filename_)
                        # keep_name_only(x)
                        
                      


In [ ]:
# TEST

# filename = '/scratch2/Dropbox/Dropbox/0_phd/tree_annotationsAug2019/annotations/temp/T46NHL_palm_Stephane.kml'
# dst_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/manual_annotations'

# split_kml_folders(filename,dst_path=dst_path,checkonly=False)

In [ ]:
src_path = '/scratch2/Dropbox/Dropbox/0_phd/tree_annotationsAug2019/annotations/Checked/'
dst_path = '/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/manual_annotations'

filelist= glob.glob(src_path+'/**/*.kml', recursive=True)

for file in filelist:
    #print(file)
    if not '_check' in file:
        split_kml_folders(file,dst_path=dst_path,checkonly=False)
    else:
        print('skipping', file)


    

In [ ]:

def get_featname(file, is_assert=False):
    if os.path.isfile:
        files = [file]
    else:
        files = glob.glob(file+'/*.shp')
    out = []
    for file in files:
        data = ogr.Open(file)
        for layer in data:
            layer.ResetReading()
            for feature in layer:
                items_ = feature.items()
                geom = feature.geometry()
                geomtype = geom.GetGeometryName()
                key_ = items_['Name'] if items_['Name'] is not None else layer.GetDescription()
                out.append({'Name':key_.lower(),
                            'geom':geomtype,
                           'file':file})
        if is_assert:
            geom_types = set([x['geom'] for x in out])
            assert len(geom_types) <=1,f'{file} has more than one geom type: {geom_types}'
            pos_names = set([x for x in out if 'pos' in x['Name'] and 'GEOM' in x['geom']])
            neg_names = set([x for x in out if 'neg' in x['Name'] and 'GEOM' in x['geom']])
            assert len(pos_names)+len(neg_names) <=1,f'{file} has both positive and negative geometries'
    return out


In [ ]:
filelist= glob.glob(dst_path+'/*/*')

for file in filelist:
    featnames_ = get_featname(file, is_assert=False)
    # print(len(featnames_),file)
    if len([x['Name'] for x in featnames_ if 'pos' in x['Name']]) == 1:
        print(f'more than one pos area in {file}')
    

#featnames = [get_featname(file, is_assert=False) for file in filelist] # TODO fix if there is more than 1 feature in .shp



In [ ]:
"cd /scratch/andresro/leon_work/barry_palm/data/labels/manual_annotations"
"ogrmerge.py -single -o coconut_merged.shp */coconut*/pos*.shp */coconut*/Pos*.shp -overwrite_ds"

In [ ]:
"ogrmerge.py -single -o palm_merged.shp */palm*/pos*.shp */palm*/Pos*.shp -overwrite_ds"

OLD

In [ ]:
# dst = path + "/kml"  # Output directory
if not os.path.isdir(dst_path):     os.mkdir(dst_path)
new_feat = ogr.Feature(layer.GetLayerDefn())  # Dummy feature

for id, feat in enumerate(layer):
    filename_ = os.path.join(dst_path, '{}_{}_{}.kml'.format(feat.GetField(6),feat.GetField(8), feat.GetField(7)))

    if not os.path.isfile(filename_):
        new_ds = driver.CreateDataSource(filename_)
        new_lyr = new_ds.CreateLayer('feat_{}'.format(id), sr, ogr.wkbPolygon)  # You have to specify the geometry type the layer will contain here with an ogr constant. I assume it is polygon but it can be changed.
        geom = feat.geometry().Clone()
        new_feat.SetGeometry(geom)
        new_lyr.CreateFeature(new_feat)

        del new_ds, new_lyr

In [ ]:

def convert_to_shp(points, is_overwrite=False):
    if points.endswith('.kml'):
        new_points=points.replace('.kml','.shp')
        if not os.path.exists(new_points) or is_overwrite:
            srcDS = gdal.OpenEx(points)
            ds = gdal.VectorTranslate(new_points, srcDS, format='ESRI Shapefile')
            ds = None
            points = new_points
    return points 


def keep_name_only(points):
    dataSource = ogr.Open(points, 1) 

    layer = dataSource.GetLayer()

    lyrdf = layer.GetLayerDefn()

    id_Name = lyrdf.GetFieldIndex('Name')
    attr_N = lyrdf.GetFieldCount()
    print(attr_N, id_Name)
    for i in range(attr_N):
        if not i == id_Name:
            layer.DeleteField(i)   
    attr_N = lyrdf.GetFieldCount()
    print(attr_N)
    dataSource = None

    
def get_name_wkt(file):
    data = ogr.Open(file)
    
    out= {}
    for layer in data:
        layer.ResetReading()
        for feature in layer:
            items_ = feature.items()
            geom = feature.geometry()
            out[items_['Name']] = geom.ExportToWkt()
#             print(geom)
#             print('Feature Geometry:', feature.geometry())
    return out

In [ ]:
tile='*'
# PATH='/scratch/andresro/leon_igp'
PATH='/home/pf/pfstaff/projects/andresro'

filelist = glob.glob(PATH+'/barry_palm/data/labels/palm_annotations/{}/**/*.kml'.format(tile))

dirnames = list({os.path.dirname(x) for x in filelist})
print(len(dirnames))

In [ ]:
file_ = '/scratch2/Dropbox/Dropbox/0_phd/tree_annotationsAug2019/annotations/temp/T46NHL_palm_Stephane.kml'
convert_to_shp(file_)
keep_name_only(file_)

In [ ]:
for file in filelist:
    file1 = convert_to_shp(file, is_overwrite=False)
    keep_name_only(file1)

In [ ]:
folder='/scratch/andresro/leon_work/barry_palm/data/labels/palm_annotations/T50NQL/group1'


filelist = glob.glob(folder+'/*.shp')

featnames = [get_featname(file, is_assert=True)[0] for file in filelist] # TODO fix if there is more than 1 feature in .shp
# pos_shp = [x for x,names in zip(filelist,featnames) if 'pos' in names[0] and 'POLYGON' in names[1]]

In [ ]:
poly_feat = ['POLY' in x[1] for x in featnames[2]]


In [ ]:

featnames

In [ ]:
set([x['geom'] for x in featnames[6]])

In [ ]:

pos_names = set([x for x in featnames[7] if 'pos' in x['Name'] and 'GEOM' in x['geom']])
neg_names = set([x for x in featnames[7] if 'neg' in x['Name'] and 'GEOM' in x['geom']])
